In [4]:
pip install pandas geopandas numpy plotly requests openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pandas as pd
import geopandas as gpd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests
from io import StringIO

# Download and prepare the datasets
def get_displacement_data():
    """Get displacement data from IDMC"""
    try:
        # Using IDMC's public API data endpoint
        displacement_url = "https://api.idmcdb.org/api/displacement_data"
        response = requests.get(displacement_url)
        if response.status_code == 200:
            df_displacement = pd.DataFrame(response.json()['results'])
        else:
            # Fallback to sample data if API is unavailable
            df_displacement = pd.DataFrame({
                'iso3': ['IND', 'CHN', 'PHL', 'USA', 'BGD'],
                'country_name': ['India', 'China', 'Philippines', 'United States', 'Bangladesh'],
                'year': [2022] * 5,
                'displacement_value': [5000000, 2500000, 2000000, 1500000, 1000000]
            })
        
        return df_displacement
    except Exception as e:
        print(f"Error fetching displacement data: {e}")
        return None

def get_climate_vulnerability_data():
    """Get climate vulnerability data"""
    try:
        # Using World Bank Climate Change Knowledge Portal API
        vulnerability_url = "https://climateknowledgeportal.worldbank.org/api/data/get-download-data"
        params = {
            'type': 'vulnerability',
            'format': 'csv'
        }
        response = requests.get(vulnerability_url, params=params)
        if response.status_code == 200:
            df_vulnerability = pd.read_csv(StringIO(response.text))
        else:
            # Fallback to sample data if API is unavailable
            df_vulnerability = pd.DataFrame({
                'Country': ['India', 'China', 'Philippines', 'United States', 'Bangladesh'],
                'Vulnerability_Index': [0.8, 0.6, 0.7, 0.3, 0.9]
            })
        
        return df_vulnerability
    except Exception as e:
        print(f"Error fetching vulnerability data: {e}")
        return None

def create_migration_hotspot_map():
    """Create interactive map of climate migration hotspots"""
    try:
        # Get world geometry data from Natural Earth
        world = gpd.read_file(
            "https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip"
        )
        
        # Get displacement and vulnerability data
        df_displacement = get_displacement_data()
        df_vulnerability = get_climate_vulnerability_data()
        
        if df_displacement is None or df_vulnerability is None:
            raise ValueError("Could not fetch required data")
        
        # Aggregate displacement data by country
        df_displacement_agg = df_displacement.groupby('country_name')['displacement_value'].sum().reset_index()
        
        # Merge data
        merged_data = world.merge(
            df_displacement_agg, 
            how='left', 
            left_on='NAME', 
            right_on='country_name'
        )
        merged_data = merged_data.merge(
            df_vulnerability, 
            how='left', 
            left_on='NAME', 
            right_on='Country'
        )
        
        # Handle duplicate values in vulnerability index
        # First, add small random noise to break ties
        merged_data['Vulnerability_Index'] = merged_data['Vulnerability_Index'].fillna(0)
        merged_data['Vulnerability_Index_jittered'] = merged_data['Vulnerability_Index'] + \
            np.random.normal(0, 1e-10, size=len(merged_data))
        
        # Calculate risk levels using the jittered values
        try:
            merged_data['RiskLevel'] = pd.qcut(
                merged_data['Vulnerability_Index_jittered'],
                q=5,
                labels=['1', '2', '3', '4', '5']
            )
        except ValueError:
            # If qcut still fails, use cut instead
            merged_data['RiskLevel'] = pd.cut(
                merged_data['Vulnerability_Index'],
                bins=5,
                labels=['1', '2', '3', '4', '5']
            )
        
        # Create interactive map
        fig = px.choropleth(
            merged_data,
            locations='ISO_A3',
            color='RiskLevel',
            hover_name='NAME',
            hover_data={
                'displacement_value': ':,.0f',
                'Vulnerability_Index': ':.3f',
                'RiskLevel': False
            },
            color_discrete_map={
                '1': '#fcbba1',
                '2': '#fc9272',
                '3': '#fb6a4a',
                '4': '#ef3b2c',
                '5': '#67000d'
            },
            title='Climate Migration Hotspots'
        )
        
        # Add displacement circles only for countries with data
        mask = merged_data['displacement_value'].notna()
        displacement_scale = merged_data.loc[mask, 'displacement_value'].max() / 1000000
        
        fig.add_trace(go.Scattergeo(
            lon=merged_data.loc[mask, 'geometry'].centroid.x,
            lat=merged_data.loc[mask, 'geometry'].centroid.y,
            text=merged_data.loc[mask, 'NAME'],
            mode='markers',
            marker=dict(
                size=np.sqrt(merged_data.loc[mask, 'displacement_value']) / np.sqrt(displacement_scale),
                color='rgba(255, 255, 255, 0.5)',
                line=dict(color='rgba(0, 0, 0, 0.2)', width=0.5)
            ),
            name='Displacement Volume'
        ))
        
        # Update layout
        fig.update_layout(
            title_x=0.5,
            geo=dict(
                showframe=False,
                showcoastlines=True,
                projection_type='equirectangular'
            ),
            height=800
        )
        
        return fig
    except Exception as e:
        print(f"Error creating map: {e}")
        return None

def analyze_trends():
    """Analyze displacement trends over time"""
    try:
        df_displacement = get_displacement_data()
        
        if df_displacement is None:
            raise ValueError("Could not fetch displacement data")
        
        # Calculate yearly trends
        yearly_trends = df_displacement.groupby('year')['displacement_value'].sum().reset_index()
        
        # Create trend visualization
        fig = px.line(
            yearly_trends, 
            x='year', 
            y='displacement_value',
            title='Global Climate Displacement Trends'
        )
        
        fig.update_layout(
            xaxis_title="Year",
            yaxis_title="Number of Displacements",
            height=500
        )
        
        return fig
    except Exception as e:
        print(f"Error analyzing trends: {e}")
        return None

def generate_report():
    """Generate comprehensive analysis report"""
    try:
        df_displacement = get_displacement_data()
        df_vulnerability = get_climate_vulnerability_data()
        
        if df_displacement is None or df_vulnerability is None:
            raise ValueError("Could not fetch required data")
        
        # Calculate key statistics
        total_displaced = df_displacement['displacement_value'].sum()
        most_affected_countries = df_displacement.groupby('country_name')['displacement_value'].sum().nlargest(10)
        most_vulnerable_countries = df_vulnerability.nlargest(10, 'Vulnerability_Index')
        
        report = f"""
        Climate Migration Hotspot Analysis Report
        
        Overall Statistics:
        - Total Recorded Displacements: {total_displaced:,.0f}
        
        Most Affected Countries (Total Displacements):
        {most_affected_countries.to_string()}
        
        Most Vulnerable Countries:
        {most_vulnerable_countries.to_string()}
        """
        
        return report
    except Exception as e:
        print(f"Error generating report: {e}")
        return None

def main():
    """Main execution function"""
    try:
        # Create visualizations
        print("Creating hotspot map...")
        hotspot_map = create_migration_hotspot_map()
        if hotspot_map:
            hotspot_map.write_html("climate_migration_hotspots.html")
            print("- Created climate_migration_hotspots.html")
        
        print("Analyzing trends...")
        trend_chart = analyze_trends()
        if trend_chart:
            trend_chart.write_html("displacement_trends.html")
            print("- Created displacement_trends.html")
        
        print("Generating report...")
        report = generate_report()
        if report:
            with open("climate_migration_report.txt", "w") as f:
                f.write(report)
            print("- Created climate_migration_report.txt")
        
        print("\nAnalysis complete.")
    except Exception as e:
        print(f"Error in main execution: {e}")

if __name__ == "__main__":
    main()

Creating hotspot map...


c:\Users\Owner\Desktop\SI_618_WN_24_Files\.venv\Lib\site-packages\plotly\express\_core.py:2044: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

c:\Users\Owner\Desktop\SI_618_WN_24_Files\.venv\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

C:\Users\Owner\AppData\Local\Temp\ipykernel_34908\4056096356.py:136: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


C:\Users\Owner\AppData\Local\Temp\ipykernel_34908\4056096356.py:137: UserWarning:

Geometry is in a geographic CRS.

- Created climate_migration_hotspots.html
Analyzing trends...
- Created displacement_trends.html
Generating report...
- Created climate_migration_report.txt

Analysis complete.
